# Detecting audio issues in the Common Voice dataset
This notebook aims at showing how you can leverage sliceguard to detect issues in audio datasets, using the commonvoice dataset as an example. Focus will be on the basic workflow, as well as showing how to leverage different embedding models from the huggingface hub.

In order to run this example you will need some **dependencies**. Install them as follows.

In [32]:
!pip install sliceguard librosa soundfile datasets tqdm jiwer

You should consider upgrading via the '/home/daniel/code/sliceguard/.venv/bin/python -m pip install --upgrade pip' command.


## Step 1: Generate predictions for the Common Voice dataset

**IMPORTANT NOTE**: In order to access the commonvoice dataset you have to accept certain terms and conditions. To do this, create a huggingface account and accept the terms and conditions [HERE](https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0). You then need to **create an access token** to access your datasets programmatically. Follow the steps for configuring one [HERE](https://huggingface.co/docs/hub/security-tokens). It is just a matter of few minutes. Just paste your access token into a file called **access_token.txt** and place it in the same directory as this notebook.

In [1]:
# Configure this example here.
# Like this it is optimized for fast execution only using whisper tiny.
HF_MODEL = "openai/whisper-tiny"
ACCESS_TOKEN_FILE = "access_token.txt"
AUDIO_SAVE_DIR = "audios"
NUM_SAMPLES = 2000

In [2]:
# Some imports your will need to execute this
import uuid
import shutil
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import torch
import librosa
import soundfile as sf
from jiwer import wer
from datasets import load_dataset, Audio
from transformers import pipeline
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer, WhisperForConditionalGeneration

In [3]:
# Read the acces token for downloading the dataset
access_token = Path(ACCESS_TOKEN_FILE).read_text()
cv_13 = load_dataset("mozilla-foundation/common_voice_13_0", "en", use_auth_token=access_token, streaming=True)

In [4]:
# Instantiate an ASR pipeline with the configured model
device = "cuda:0" if torch.cuda.is_available() else "cpu"

feature_extractor = WhisperFeatureExtractor.from_pretrained(HF_MODEL)
tokenizer = WhisperTokenizer.from_pretrained(HF_MODEL, language="en", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(HF_MODEL).to(device)

model.config.forced_decoder_ids = tokenizer.get_decoder_prompt_ids() # Specify the task as we always want to use german and transcribe
model.config.language = "<|en|>"
model.config.task = "transcribe"

pipe = pipeline("automatic-speech-recognition", model=model, tokenizer=tokenizer, feature_extractor=feature_extractor, device=device)

In [5]:
keys_to_save = ["sentence", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "variant"]

audio_save_dir = Path(AUDIO_SAVE_DIR)
if  not audio_save_dir.is_dir():
    audio_save_dir.mkdir()
else:
    shutil.rmtree(audio_save_dir)
    audio_save_dir.mkdir()

num_samples = 0
data = []
for sample in tqdm(cv_13["train"], total=NUM_SAMPLES):
    new_audio = librosa.resample(sample["audio"]["array"], orig_sr=sample["audio"]["sampling_rate"], target_sr=16000)
    file_stem = str(uuid.uuid4())
    cur_data = {}
    for k in keys_to_save:
        cur_data[k] = sample[k]
    prediction = pipe(new_audio)["text"]
    cur_data["prediction"] = prediction
    
    sample_wer = wer(sample["sentence"], prediction)
    cur_data["wer"] = sample_wer
    
    target_path = audio_save_dir / (file_stem + ".wav")
    cur_data["audio"] = target_path
    sf.write(target_path, new_audio, 16000)
    data.append(cur_data)
    num_samples += 1
    if num_samples > NUM_SAMPLES:
        break

  0%|                                                 | 0/2000 [00:00<?, ?it/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 1it [00:01,  1.68s/it]
Reading metadata...: 22068it [00:03, 8017.60it/s]
Reading metadata...: 22891it [00:03, 6430.01it/s]
Reading metadata...: 43987it [00:05, 9166.01it/s]
Reading metadata...: 65891it [00:07, 10854.95it/s]
Reading metadata...: 87679it [00:08, 11906.11it/s]
Reading metadata...: 109202it [00:10, 11952.33it/s]
Reading metadata...: 130629it [00:12, 12475.19it/s]
Reading metadata...: 151838it [00:12, 18010.87it/s]
Reading metadata...: 159303it [00:13, 13138.15it/s]
Reading metadata...: 172914it [00:15, 11057.68it/s]
Reading metadata...: 193788it [00:15, 17035.31it/s]
Reading metadata...: 202868it [00:18, 9979.08it/s] 
Reading metadata...: 214827it [00:19, 9327.82it/s]
Reading metadata...: 235814it [00:21, 10136.95it/s]
Reading metadata...: 256646it [00:23, 11144.02it/s]
Reading metadata...: 277374it [00:24, 11797.52it/s]
Reading metad

In [6]:
df = pd.DataFrame(data)
df["audio"] = df["audio"].astype("string") # otherwise overflow in serializing json
df.to_json("dataset.json", orient="records")

## Step 2: Detect issues caused by environmental noise
First check we want to do is checking wether there are audio recordings that are somehow so different from the rest of the data that they cannot be properly transcribed. Here we mostly target **general audio properties and environmental noise** such as background noises.

In order to do this, we leverage **general purpose audio embeddings** of a model trained on Audioset.

In [7]:
# Some imports you will need for this step
import pandas as pd
import numpy as np
from jiwer import wer
from sliceguard import SliceGuard
from renumics.spotlight import Audio

In [8]:
# Read the generated dataset including the predictions
df = pd.read_json("dataset.json")

In [9]:
# Define the metric function
def wer_metric(y_true, y_pred):
    return np.mean([wer(s_y, s_pred) for s_y, s_pred in zip(y_true, y_pred)])

In [33]:
# Perform an initial detection aiming for relatively small clusters of minimum 5 similar samples
sg = SliceGuard()
issue_df = sg.find_issues(
        df,
        ["audio"],
        "sentence",
        "prediction",
        wer_metric,
        metric_mode="min",
        embedding_models={"audio": "MIT/ast-finetuned-audioset-10-10-0.4593"},
        min_support=5,
        min_drop=0.2,
    )

Feature audio was inferred as referring to raw data. If this is not the case, please specify in feature_types!


Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.dense.bias', 'classifier.layernorm.weight', 'classifier.layernorm.bias', 'classifier.dense.weight']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████| 2001/2001 [02:01<00:00, 16.43it/s]


The overall metric value is 0.3165315929558808
Using 5 as minimum support for determining problematic clusters.
Using 0.2 as minimum drop for determining problematic clusters.
Identified 7 problematic slices.


In [34]:
# Report the issues using Renumics Spotlight
sg.report(spotlight_dtype={"audio": Audio})

In [35]:
# Of course if you want to run additional checks you don't need to recompute the embeddings all the time.
# Just save them here, and supply the precomputed embeddings in the next call
# where we will check for smaller clusters aka outliers.
computed_embeddings = sg.embeddings

In [36]:
# Perform an additional detection, targeting outliers with significant drops (see min_support and min_drop)
# We even allow for clusters containing single samples here.
sg = SliceGuard()
issue_df = sg.find_issues(
        df,
        ["audio"],
        "sentence",
        "prediction",
        wer_metric,
        metric_mode="min",
        min_support=1,
        min_drop=0.5,
        precomputed_embeddings=computed_embeddings
    )

Feature audio was inferred as referring to raw data. If this is not the case, please specify in feature_types!
The overall metric value is 0.3165315929558808
Using 1 as minimum support for determining problematic clusters.
Using 0.5 as minimum drop for determining problematic clusters.
Identified 7 problematic slices.


In [37]:
# Report the issues using Renumics Spotlight
sg.report(spotlight_dtype={"audio": Audio})

## Step 3: Detect issues caused by (uncommon) speakers
While the previous detection example targeted finding general audio conditions that can cause issues, this is not always the criterion we want to check for. A way of defining other criterions is **changing the underlying embedding** to **capture different properties of the data**. In this case, we define the embedding model to be a model for **speaker identification**. This should allow us, to **detect uncommon speakers**, although they are note explicitely labeled.

In [22]:
# Perform a detection using a speaker identification model for computing embeddings.
# This will help to recover problematic speakers even though they are not explicitely labeled.
sg = SliceGuard()
issue_df = sg.find_issues(
        df,
        ["audio"],
        "sentence",
        "prediction",
        wer_metric,
        metric_mode="min",
        embedding_models={"audio": "superb/wav2vec2-base-superb-sid"},
        min_support=3,
        min_drop=0.4,
    )

Feature audio was inferred as referring to raw data. If this is not the case, please specify in feature_types!


/home/daniel/code/sliceguard/.venv/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at superb/wav2vec2-base-superb-sid were not used when initializing Wav2Vec2Model: ['layer_weights', 'classifier.weight', 'classifier.bias', 'projector.weight', 'projector.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a

The overall metric value is 0.3165315929558808
Using 3 as minimum support for determining problematic clusters.
Using 0.4 as minimum drop for determining problematic clusters.
Identified 4 problematic slices.


In [23]:
# Report the issues using Renumics Spotlight
sg.report(spotlight_dtype={"audio": Audio})